In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
# Load, process, and save a Roli Block performance.
data = pd.read_csv("../datasets/2018-06-21-block-perf1.txt")
data['dt'] = data.time.diff().fillna(0)

display(data.columns)

data['x'] = data.message.apply((lambda x: int(x.split(' ')[2])))
data['y'] = data.message.apply((lambda x: int(x.split(' ')[3])))
data['z'] = data.message.apply((lambda x: int(x.split(' ')[4])))

cleaned_data = data[['dt', 'x', 'y', 'z']]
display(cleaned_data.describe())

array_version = np.array(data[['x','y','z']])
display(array_version.shape)

# Test saving
np.savez('roli-block-session-data.npz', array_version)

In [ ]:
# Test loading
with np.load('roli-block-session-data.npz') as data:
    dataset = data['arr_0']
dataset.shape

In [ ]:
# Look at Gesture-RNN version:
# URL: https://github.com/lamtharnhantrakul/GestureRNN-ML4Lightpad/blob/master/data/test_data.txt
# Download (if needed)
import urllib.request
url = 'https://github.com/lamtharnhantrakul/GestureRNN-ML4Lightpad/raw/master/data/test_data.txt'
urllib.request.urlretrieve(url, './gesture-rnn-data.txt')
# Parse the file
grnn_df = pd.read_csv('./gesture-rnn-data.txt', delim_whitespace=True, header=None, lineterminator=';')
grnn_df = grnn_df[[1,2,3]]
grnn_df.columns = ['x','y','z']
# Prove it works
display(grnn_df.head())
# Do some analysis
display(grnn_df.describe())

grnn_array = np.array(grnn_df)
display(grnn_array.shape)
np.savez('grnn-data.npz', grnn_array)


In [ ]:
# Test loading
with np.load('grnn-data.npz') as data:
    dataset = data['arr_0']
dataset.shape

In [ ]:
import keras

# Training Hyperparameters:
SEQ_LEN = 30
BATCH_SIZE = 256
HIDDEN_UNITS = 64
EPOCHS = 30
VAL_SPLIT=0.2

# These settings train for 2.1 epochs which is pretty good!
SEED = 2345  # 2345 seems to be good.
np.random.seed(SEED)
# tf.set_random_seed(5791)  # only works for current graph.

encoder = keras.Sequential()
encoder.add(keras.layers.LSTM(HIDDEN_UNITS, batch_input_shape=(None,SEQ_LEN,3), return_sequences=True))
encoder.add(keras.layers.LSTM(HIDDEN_UNITS))
encoder.add(keras.layers.Dense(3, activation='relu'))
encoder.compile(loss='mse', optimizer=keras.optimizers.Adam())
encoder.summary()

In [ ]:
def slice_sequence_examples(sequence, num_steps):
    xs = []
    for i in range(len(sequence) - num_steps - 1):
        example = sequence[i: i + num_steps]
        xs.append(example)
    print("Total training examples:", str(len(xs)))
    return xs

def seq_to_singleton_format(examples):
    """
    Return the examples in seq to singleton format.
    """
    xs = []
    ys = []
    for ex in examples:
        xs.append(ex[:-1])
        ys.append(ex[-1])
    return (xs,ys)


X, y = seq_to_singleton_format(slice_sequence_examples(grnn_array, SEQ_LEN+1))
X = np.array(X)
y = np.array(y)

print("X:", X.shape)
print("y:", y.shape)

In [ ]:
history = encoder.fit(X, y, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=VAL_SPLIT)

In [ ]:
X, y = seq_to_singleton_format(slice_sequence_examples(array_version, SEQ_LEN+1))
X = np.array(X)
y = np.array(y)

In [ ]:
array_version = array_version

In [ ]:
array_version = array_version / 128
array_version

# Comments!

Interesting, it looks as if the GRNN data is very easy to train, loss below 0.01 in 30 epochs (30 seconds) of training.

What kind of analysis can be done to investigate this?